In [ ]:
#importing stuff and setting global variables
import cv2 as cv# for realtime processing
import math
import mediapipe as mp# fro AI 
#import pyautogui as py#yeh screenshot k liay hai
# from mediapipe_model_maker import gesture_recognizer

The Code below i for creating the task or i would say a medol instance for recognizing guestures for a curstom data set
First piece of code is to load data from a folder into variable data and than split it into testing and training data
Next peice of code of is Training data than testing data data and finally exporting the model 

In [ ]:
# IMAGES_PATH = "rps_data_sample"
# data = gesture_recognizer.Dataset.from_folder(
#     dirname=IMAGES_PATH,
#     hparams=gesture_recognizer.HandDataPreprocessingParams()
# )
# # Split the archive into training, validation and test dataset.
# train_data, rest_data = data.split(0.8)
# validation_data, test_data = rest_data.split(0.5)

In [ ]:
# hparams = gesture_recognizer.HParams(export_dir="rock_paper_scissors_model")
# options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
# model = gesture_recognizer.GestureRecognizer.create(
#     train_data=train_data,
#     validation_data=validation_data,
#     options=options
# )

In [ ]:
# loss, acc = model.evaluate(test_data, batch_size=1)
# print(f"Test loss:{loss}, Test accuracy:{acc}")

In [ ]:
# model.export_model()

The code below is for drawing landmarks on real time frame

In [ ]:
mp_holistic = mp.solutions.holistic

mp_drawing = mp.solutions.drawing_utils
def hand_detector(frame,holistic):
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)

    ## for drawing left hand
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))
        
    ## From drawing right hand
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))
    
        
    return frame

The code bellow is load the model and use it for real-time and making it ready for real time camera application

In [ ]:
BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
GestureRecognizerResult = mp.tasks.vision.GestureRecognizerResult
VisionRunningMode = mp.tasks.vision.RunningMode
result_name = [None,0]
def print_result(result: GestureRecognizerResult, output_image: mp.Image, timestamp_ms: int):
     #print('gesture recognition result: {}'.format(result))
     if len(result.gestures) and len(result.gestures[0]):
      gesture = result.gestures[0][0]
      if result_name[0] != gesture.category_name:
         result_name[0] = gesture.category_name
         result_name[1] = 0
      else:
         result_name[1] += 1
      print(gesture.category_name,result_name[1])
        
     else:
        result_name[0] = None
        result_name[1] = 0



    

options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path='exported_model/gesture_recognizer.task'),
    running_mode=VisionRunningMode.LIVE_STREAM,result_callback=print_result
    )
recognizer =  GestureRecognizer.create_from_options(options) 
timestamp = 0

In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.8)
cap = cv.VideoCapture(0)

cap.set(cv.CAP_PROP_FRAME_HEIGHT,720)
cap.set(cv.CAP_PROP_FRAME_WIDTH,1080)
while cap.isOpened():
    timestamp += 1
    ret, frame = cap.read()
    if not ret:
        continue

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    recognition_result = recognizer.recognize_async(mp_image,timestamp)
    #print(result_name[0])

    frame = hand_detector(frame,holistic)

    cv.imshow("Hand Sign Detection", frame )    
    if cv.waitKey(1) & 0xFF == ord('q') or (result_name[0] == "scissors" and result_name[1] > 6):
        break

    if result_name[0] == "paper" and result_name[1] > 6:
        screenshot = py.screenshot()
        screenshot.save("screenshot.png")
    
        
cap.release()
cv.destroyAllWindows()
